# 7. デプロイ済みのCMLモデルを利用する二つの方法

CML でホストされたモデルを利用するには、二つの方法があります。
一つは cmlapi ライブラリを利用する方法です。これは、[APIv2インターフェース](https://docs.cloudera.com/machine-learning/1.5.4/rest-api-reference/index.html#/CMLService)の Python ラッパーです。
もうひとつの方法は、HTTPリクエストを直接投げることです。

どちらの方法でも、呼び出し元の認証を容易にし、メッセージ交換形式として JSON を使用できます。

このノートブックでは、cmlapi 経由で必要な変数を収集した上で、requests Python パッケージを使用してモデルエンドポイントにリクエストを行っています。

### 7.1 CML client を起動する
CMLの API に関する詳細なドキュメントは、 [こちら](https://docs.cloudera.com/machine-learning/cloud/api/topics/ml-apiv2-usage-examples.html) を参照してください。

In [ ]:
import cmlapi
import json
import sys
import os
client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
client.list_projects()

### 7.2 CML プロジェクトの検索

このハンズオンでは、今このファイルがあるプロジェクトとは別のプロジェクトでモデルをホストしています。
その（モデルをホストしているほうの）プロジェクトIDを、名前で検索して取得することができます。

In [ ]:
## プロジェクトの中にモデルがひとつだけデプロイされていることを想定したコードです。
## 複数のモデルがある場合は、project.projects[x] のインデックスを調整します。

projects = client.list_projects(include_public_projects=True, search_filter=json.dumps({"name": "LLM_local_model"}))
project = projects.projects[0]
print(project)

### 7.3 CML モデルのアクセスキーを取得し、環境変数に保管する

In [3]:
## プロジェクトの中にモデルがひとつだけデプロイされていることを想定したコードです。
## 複数のモデルがある場合は、 model.models[x] のインデックスを調整します。
model = client.list_models(project_id=project.id)
selected_model = model.models[0]

## モデルのアクセスキーを、このプロジェクトの環境変数として保存します。
os.environ["MODEL_ACCESS_KEY"] = selected_model.access_key

In [ ]:
# モデルのアクセスキーを表示（確認用）
os.environ["MODEL_ACCESS_KEY"]

### 7.5 リクエスト用のモデルのエンドポイントを生成する
取得した環境変数を編集して、リクエスト用のモデルのエンドポイントを作成します。

In [5]:
MODEL_ENDPOINT = os.getenv("CDSW_API_URL").replace("https://", "https://modelservice.").replace("/api/v1", "/model?accessKey=")
MODEL_ENDPOINT = MODEL_ENDPOINT + os.environ["MODEL_ACCESS_KEY"]
os.environ["MODEL_ENDPOINT"] = MODEL_ENDPOINT

In [ ]:
# モデルのエンドポイントを表示（確認用）
os.environ["MODEL_ENDPOINT"]

### 7.6 モデルへのリクエストを作成する
最後に、上記の変数を使って実際にリクエストを投げてみましょう。

In [ ]:
import requests
import json
import os

## 変数の設定
temperature = 0.01
token_count = 150

## モデルに聞く質問を設定
question = "りんごの色は何色ですか？"
context = "知っている知識の範囲で回答し、知らない場合は「知らない」と回答してください。日本語でお願いします。"

question_and_context = question + context

data={ "request": {"prompt":question_and_context,"temperature":temperature,"max_new_tokens":token_count,"repetition_penalty":1.0} }

r = requests.post(os.environ["MODEL_ENDPOINT"], data=json.dumps(data), headers={'Content-Type': 'application/json'})

# 内容の表示
print(f"Request: {data} \n\n")
print(f"Response: {r.json()}")

### 7.7 要約

* CMLにデプロイされたモデルは、JSONペイロードを持つAPIのエンドポイントとしてコールできます。
* ユースケースに応じて、モデルは負荷を許容するために複数のレプリカを持つことができます。
* 認証はモデルのアクセスキーと、（必要に応じて）ユーザーのアクセスキーによって行われます。